# Importing Packages

In [1]:
!pip install braindecode
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.2/165.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.1 MB/s eta 0:00:00


In [2]:
import braindecode

In [ ]:
import mne
from scipy.io import loadmat
import scipy
import sklearn
import numpy as np
import pandas as pd
import glob
from mne.decoding import CSP
import os

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda

In [ ]:
import warnings
warnings.filterwarnings('ignore') # to ignore warnings

In [ ]:
verbose = False                    # global variable to suppress output display of MNE functions
mne.set_log_level(verbose=verbose) # to suppress large info outputs

In [ ]:
n_jobs = None  # for multicore parallel processing, set it to 1 if cause memory issues, for full utilization set to -1

# Data Loading and Conversion to MNE Datatypes


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_path = "/content/drive/MyDrive/GP/Data"

In [ ]:
training_files   = glob.glob(folder_path + '/*T.mat')
len(training_files)     # if  return zero,then no file is loaded

10

In [ ]:
# we have modified the labels values from [1, 2] to [0, 1] as pytorch
# expects labels/classes to be in [0, n_classes-1] format
def get_mne_epochs(filepath, verbose=verbose, t_start=2, fs=512, mode='train'):
    '''
    This function reads the EEG data from .mat file and convert it to MNE-Python Compatible epochs
    data structure. It takes data from [0, 8] sec range and return it by setting t = 0 at cue onset
    i.e. 3 seconds and dropping first two seconds so the output data is in [-1.0, 5.0] sec range. The
    Details can be found in the preprocessing section of the attached document
    '''
    mat_data = loadmat(filepath) # read .mat file
    eeg_data= mat_data['RawEEGData']
    idx_start = fs*t_start
    eeg_data = eeg_data[:, :, idx_start:]
    event_id = {'left-hand': 0, 'right-hand': 1} # pytorch expects labels in [0, n_classes-1]
    channel_names = ['F3', 'FC3', 'C3', 'CP3', 'P3', 'FCz', 'CPz', 'F4', 'FC4', 'C4', 'CP4', 'P4']
    info = mne.create_info(ch_names=channel_names, sfreq=fs, ch_types='eeg')
    epochs = mne.EpochsArray(eeg_data, info, verbose=verbose, tmin=t_start-3.0)
    epochs.set_montage('standard_1020')
    epochs.filter(1., None)
    epochs.apply_baseline(baseline=(-.250, 0)) # linear baseline correction

    if mode == 'train': # this in only applicable for training data
        epochs.event_id = event_id
        epochs.events[:,2] = mat_data['Labels'].ravel() - 1
    return epochs

def get_labels(filepath):
    mat_data = loadmat(filepath) # read .mat file
    return mat_data['Labels'].ravel() - 1

In [ ]:
epochs, labels = get_mne_epochs(training_files[0], verbose=verbose), get_labels(training_files[0])
data = epochs.get_data()
print('Shape of EEG Data: ', data.shape, '\t Shape of Labels: ', labels.shape)

Shape of EEG Data:  (80, 12, 3072) 	 Shape of Labels:  (80,)


## Data Processing

In [ ]:
# loading original data
epochs_list = []
for i in training_files:
    epochs_list.append(get_mne_epochs(i, verbose=verbose))

In [ ]:
len(epochs_list)

10

In [ ]:
# split epochs_list 60 trials train and 20 trials test
epochs_list_train = []
epochs_list_test = []
epochs_list_val = []
for epoch in epochs_list:
  # train_epoch = mne.concatenate_epochs([epoch[0:50], epoch[70:80]])
  train_epoch = epoch[0:40]
  test_epoch =  epoch[50:70]
  val_epoch = mne.concatenate_epochs([epoch[40:50], epoch[70:80]])
  epochs_list_train.append(train_epoch)
  epochs_list_test.append(test_epoch)
  epochs_list_val.append(val_epoch)

In [ ]:
len(epochs_list_test)

10

In [ ]:
epochs_list_train[0]

Number of events,40
Events,left-hand: 20right-hand: 20
Time range,-1.000 – 4.998 s
Baseline,-0.250 – 0.000 s


In [ ]:
epochs_list_test[0]

Number of events,20
Events,left-hand: 10right-hand: 10
Time range,-1.000 – 4.998 s
Baseline,-0.250 – 0.000 s


In [ ]:
epochs_list_val[0]

Number of events,20
Events,left-hand: 10right-hand: 10
Time range,-1.000 – 4.998 s
Baseline,-0.250 – 0.000 s


### [0.5, 4.5] sec and 2sec window with 125ms stride

In [ ]:
from braindecode.datautil import create_from_mne_epochs

# convert epochs to braindecode compatible datastructure
# 2sec windows with 0.125 sec stride
window_size = 1024 #512 #50 # 3072
window_stride = 64 #256 # 50

In [ ]:
windows_datasets_list_train = []
for epoch in epochs_list_train:
    windows_datasets_list_train.append(
            create_from_mne_epochs(
            [epoch.crop(tmin=0.5, tmax=4.5, include_tmax=False)], # [0.5, 4.5] s, expects list of epochs
            window_size_samples = window_size,
            window_stride_samples = window_stride,
            drop_last_window = False
        )
    )

In [ ]:
windows_datasets_list_test = []
for epoch in epochs_list_test:
    windows_datasets_list_test.append(
            create_from_mne_epochs(
            [epoch.crop(tmin=0.5, tmax=4.5, include_tmax=False)], # [0.5, 4.5] s, expects list of epochs
            window_size_samples = window_size,
            window_stride_samples = window_stride,
            drop_last_window = False
        )
    )

In [ ]:
windows_datasets_list_val = []
for epoch in epochs_list_val:
    windows_datasets_list_val.append(
            create_from_mne_epochs(
            [epoch.crop(tmin=0.5, tmax=4.5, include_tmax=False)], # [0.5, 4.5] s, expects list of epochs
            window_size_samples = window_size,
            window_stride_samples = window_stride,
            drop_last_window = False
        )
    )

In [ ]:
# windows_datasets_list_train
# windows_datasets_list_test
# windows_datasets_list_val

In [ ]:
len(windows_datasets_list_train)

10

In [ ]:
windows_datasets_list_train[0][0]

(array([[ 42.84346  ,  57.55566  ,  -6.910057 , ..., -47.189068 ,
         -84.53391  , -26.119097 ],
        [ 63.35543  ,  81.06551  ,  -1.9977584, ..., -45.45282  ,
         -81.216805 , -10.636304 ],
        [ 70.022354 ,  76.084    , -19.760208 , ..., -33.182205 ,
         -76.468445 , -16.374454 ],
        ...,
        [ 45.264866 ,  64.99564  ,   2.563443 , ..., -33.443333 ,
         -73.42125  , -15.326817 ],
        [ 50.250744 ,  76.19861  ,   5.3295617, ..., -44.18488  ,
         -88.172806 , -15.270915 ],
        [ 59.11915  ,  68.972176 ,  -2.2951732, ..., -40.723244 ,
         -77.005905 , -12.628537 ]], dtype=float32),
 0,
 [0, 0, 512])

In [ ]:
def get_windows_datasets_labels(windows_dataset):
    labels = []
    for i in range(len(windows_dataset.datasets)):
        labels.extend(windows_dataset.datasets[i].y)
    return np.array(labels)

In [ ]:
windows_datasets_labels_list_train = []
for windows_dataset in windows_datasets_list_train:
  windows_datasets_labels_list_train.append(get_windows_datasets_labels(windows_dataset))


In [ ]:
windows_datasets_labels_list_test = []
for windows_dataset in windows_datasets_list_test:
  windows_datasets_labels_list_test.append(get_windows_datasets_labels(windows_dataset))

In [ ]:
windows_datasets_labels_list_val = []
for windows_dataset in windows_datasets_list_val:
  windows_datasets_labels_list_val.append(get_windows_datasets_labels(windows_dataset))

In [ ]:
# windows_datasets_list_train
# windows_datasets_list_test
# windows_datasets_list_val
# ####
# windows_datasets_labels_list_train
# windows_datasets_labels_list_test
# windows_datasets_labels_list_val


In [ ]:
len(windows_datasets_labels_list_train)

10

In [ ]:
# windows_datasets_list_test

In [ ]:
print("Total Windows in a Train Dataset: ", len(windows_datasets_labels_list_train[0]))

Total Windows in a Train Dataset:  1000


In [ ]:
print("Total Windows in a Test Dataset: ", len(windows_datasets_labels_list_test[0]))

Total Windows in a Test Dataset:  500


In [ ]:
print("Total Windows in a Val Dataset: ", len(windows_datasets_labels_list_val[0]))

Total Windows in a Val Dataset:  500


In [ ]:
print("Total Windows in a Val Dataset: ", len(windows_datasets_list_val[0]))

Total Windows in a Val Dataset:  500


## Processing

In [ ]:
low_cut_hz = 8.  # low cut frequency for filtering
high_cut_hz = 32.  # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000

In [ ]:
from braindecode.preprocessing import (
    exponential_moving_standardize,
    preprocess,
    Preprocessor,
)

In [ ]:
preprocessors = [
    Preprocessor("pick_types", eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(
        lambda data, factor: np.multiply(data, factor),  # Convert from V to uV
        factor=1e6,
    ),
    Preprocessor("filter", l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
]

In [ ]:
for windows_dataset_train in windows_datasets_list_train:
    preprocess(windows_dataset_train, preprocessors)

In [ ]:
for windows_dataset_test in windows_datasets_list_test:
    preprocess(windows_dataset_test, preprocessors)

In [ ]:
for windows_dataset_val in windows_datasets_list_val:
    preprocess(windows_dataset_val, preprocessors)

# Data splitting and conversion

In [ ]:
windows_datasets_list_train[0][0]

(array([[ 1.38551623e-01, -1.40089203e+02, -1.28538315e+02, ...,
         -4.14386368e+01,  1.08607063e+02,  1.25540741e+02],
        [-4.40066650e+02, -5.70411926e+02, -1.55259762e+01, ...,
          2.14133011e+02,  4.53180573e+02,  1.16058907e+02],
        [-1.11367775e+02, -2.65115112e+02, -1.30591202e+02, ...,
         -1.90326691e+01,  1.63858353e+02,  1.19796089e+02],
        ...,
        [-1.25597913e+03, -1.36236462e+03,  2.66068512e+02, ...,
          8.63823792e+02,  1.27975635e+03,  1.31752991e+02],
        [-1.10894299e+03, -1.20179968e+03,  2.19152435e+02, ...,
          7.41544556e+02,  1.10100635e+03,  1.06557518e+02],
        [-4.90930206e+02, -5.83403503e+02,  3.80975800e+01, ...,
          2.63470795e+02,  4.98954041e+02,  1.09115707e+02]], dtype=float32),
 0,
 [0, 0, 512])

In [ ]:
train_windows_list = []
for subject in windows_datasets_list_train:
  subject_windows = []
  for window in subject:
    subject_windows.append(window[0])
  train_windows_list.append(np.array(subject_windows))
  print(len(subject_windows))
print(len(train_windows_list))

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
10


In [ ]:
test_windows_list = []
for subject in windows_datasets_list_test:
  subject_windows = []
  for window in subject:
    subject_windows.append(window[0])
  test_windows_list.append(np.array(subject_windows))
  print(len(subject_windows))
print(len(test_windows_list))

500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
20


In [ ]:
val_windows_list = []
for subject in windows_datasets_list_val:
  subject_windows = []
  for window in subject:
    subject_windows.append(window[0])
  val_windows_list.append(np.array(subject_windows))
  print(len(subject_windows))
print(len(test_windows_list))

500
500
500
500
500
500
500
500
500
500
20


In [ ]:
# train_windows_list
# test_windows_list
# val_windows_list

In [ ]:
# train_windows_list

In [ ]:
type(train_windows_list)

list

In [ ]:
type(train_windows_list[0])

numpy.ndarray

In [ ]:
len(train_windows_list[0])

1000

In [ ]:
print(len(train_windows_list[0]))
print(type(train_windows_list[0]))

1000
<class 'numpy.ndarray'>


In [ ]:
print(len(train_windows_list[0][0]))
print(type(train_windows_list[0][0]))

12
<class 'numpy.ndarray'>


In [ ]:
train_windows_list[0][0].shape[0]

12

In [ ]:
train_windows_list[0][0].shape[1]

512

In [ ]:
print(len(train_windows_list))
print(len(test_windows_list))

10
20


In [ ]:
print(len(train_windows_list[0]))
print(len(test_windows_list[0]))
print(len(val_windows_list[0]))

1000
500
500


## Test Split

In [ ]:
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from skorch.helper import predefined_split, SliceDataset

In [ ]:
X_train_list = train_windows_list
y_train_list = windows_datasets_labels_list_train

X_test_list = test_windows_list
y_test_list = windows_datasets_labels_list_test

X_val_list = val_windows_list
y_val_list = windows_datasets_labels_list_val

In [ ]:
print(type(X_train_list[0]))
print(type(y_train_list[0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
print(type(X_val_list[0]))
print(type(y_test_list[0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
print(type(X_val_list[0]))
print(type(y_val_list[0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
X_train_list[0].shape

(1000, 12, 512)

In [ ]:
X_test_list[0].shape

(500, 12, 512)

In [ ]:
X_val_list[0].shape

(500, 12, 512)

In [ ]:
len(y_train_list[0])

1000

In [ ]:
len(y_test_list[0])

500

In [ ]:
y_train_list[0].shape

(1000,)

In [ ]:
# X_train_list = []
# X_test_list = []
# y_train_list = []
# y_test_list = []
# for subject_idx in range(len(X_list)):
#   X_train, X_test, y_train, y_test = train_test_split(X_list[subject_idx], y_list[subject_idx], test_size=0.2, random_state=42)
#   X_train_list.append(X_train)
#   X_test_list.append(X_test)
#   y_train_list.append(y_train)
#   y_test_list.append(y_test)

In [ ]:
len(X_train_list)

10

In [ ]:
X_train_list[0].shape

(1000, 12, 512)

In [ ]:
X_val_list[0].shape

(500, 12, 512)

In [ ]:
y_val_list[0].shape

(500,)

In [ ]:
# y_val_list[0]


# Model Training

In [ ]:
# Creating a model
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet, EEGNetv4
from braindecode.models import EEGInceptionMI
# Training time
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier

In [ ]:
batch_size = 50
n_epochs = 15 #25 #20 #25 use few epochs for quick verification

In [ ]:
cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True
seed = 20200220  # random seed to make results reproducible
# Set random seed to be able to reproduce results
set_random_seeds(seed=seed, cuda=cuda)
cuda

True

In [ ]:
n_classes=2
# Extract number of chans and time steps from dataset
n_chans = train_windows_list[0][0].shape[0]
input_window_samples = train_windows_list[0][0].shape[1]

In [ ]:
print(n_chans)
print(input_window_samples)

12
512


In [ ]:
# model = EEGNetv4(
#     n_chans,
#     n_classes,
#     input_window_samples = window_size, #input_window_samples,
#     final_conv_length='auto',
# )

model = EEGInceptionMI(
    12,
    2,
    input_window_seconds = 2,
    sfreq  = 512
)

# Send model to GPU
if cuda:
    model.cuda()

In [ ]:
from skorch.dataset import Dataset

In [ ]:
val_dataset_list = []
for val_idx in range(len(X_val_list)):
  val_dataset_list.append(Dataset(X_val_list[val_idx], y_val_list[val_idx]))

In [ ]:
# val_dataset_list[0]

In [ ]:
len(val_dataset_list)

10

In [ ]:
lr = 1 * 0.02 #0.01
weight_decay = 0.5 * 0.001

clfs_list = []
for i in range(len(val_dataset_list)):
    clfs_list.append(
        EEGClassifier(
                    model,
                    criterion=torch.nn.NLLLoss,
                    optimizer=torch.optim.AdamW,
                    train_split=predefined_split(val_dataset_list[i]),  # using valid_set for validation
                    # train_split = ,  # using valid_set for validation
                    optimizer__lr=lr,
                    optimizer__weight_decay=weight_decay,
                    batch_size=batch_size,
                    callbacks=[
                        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
                    ],
                    device=device,
                    classes=n_classes,
                    max_epochs=n_epochs,
                    )
                )

In [ ]:
def training_function(subject_index):
    # dataset = windows_datasets_list[subject_index]
    clfs_list[subject_index].fit(X_train_list[subject_index], y=y_train_list[subject_index], epochs=n_epochs);
    best_validation_acc = clfs_list[subject_index].callbacks_[4][1].best_score_ # a hack to get best validation accuracy
    # best_validation_kappa = (2*best_validation_acc)-1
    print("Best Cross Validation Score: {:.2f}".format(best_validation_acc))
    test_acc = clfs_list[subject_index].score(X_test_list[subject_index], y=y_test_list[subject_index])
    print(f"Test acc: {(test_acc * 100):.2f}%")


In [ ]:
# clfs_list[0].fit(X_train_list[0], y=y_train_list[0], epochs=n_epochs)

In [ ]:
for subject_index in range(len(training_files)):
  print('\n', '#'*25, 'Training for Subject:', subject_index+1, '#'*25, '\n')
  training_function(subject_index)


 ######################### Training for Subject: 1 ######################### 

Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_accuracy    train_loss    valid_acc    valid_accuracy    valid_loss      lr      dur
-------  ----------------  ------------  -----------  ----------------  ------------  ------  -------
      1            0.9850        0.0470       0.5300            0.5300        6.4243  0.0200  35.3954
      2            0.5270        0.0622       0.5280            0.5280        5.8823  0.0197  34.9178
      3            0.7270        0.0308       0.5860            0.5860        3.1676  0.0190  35.0770
      4            0.9800        0.0243       0.6220            0.6220        5.6471  0.0178  35.1225
      5            0.9150        0.0127       0.6440            0.6440        1.7221  0.0162  35.0050
      6            1.0000        0.0074       0.6780            0.6780        2.3180  0.0143  34.9535
      7            1.0000 

In [ ]:
for subject_index in range(len(training_files)):
  print('\n', '#'*25, 'Training for Subject:', subject_index+1, '#'*25, '\n')
  training_function(subject_index)


 ######################### Training for Subject: 1 ######################### 

  epoch    train_accuracy    train_loss    valid_acc    valid_accuracy    valid_loss      lr      dur
-------  ----------------  ------------  -----------  ----------------  ------------  ------  -------
      1            0.5000        1.1430       0.5000            0.5000       58.8650  0.0200  41.6516
      2            0.5825        0.6469       0.3600            0.3600        1.8979  0.0197  35.7057
      3            0.7875        0.5536       0.5500            0.5500        0.7844  0.0190  35.8946
      4            0.6467        0.4426       0.6200            0.6200        0.8273  0.0178  35.9386
      5            0.8817        0.3442       0.5800            0.5800        0.9414  0.0162  35.9188
      6            0.6042        0.2033       0.6200            0.6200        1.7580  0.0143  35.9751
      7            0.7683        0.0811       0.4867            0.4867        2.3028  0.0122  35.9400
  

In [ ]:
for subject_index in range(len(training_files)):
  print('\n', '#'*25, 'Training for Subject:', subject_index+1, '#'*25, '\n')
  training_function(subject_index)


 ######################### Training for Subject: 1 ######################### 

Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_accuracy    train_loss    valid_acc    valid_accuracy    valid_loss      lr      dur
-------  ----------------  ------------  -----------  ----------------  ------------  ------  -------
      1            0.4943        0.8294       0.4954            0.4954       59.5727  0.0200  71.8938
      2            0.5000        0.5139       0.5092            0.5092        5.3554  0.0199  50.5486
      3            0.6690        0.3248       0.6789            0.6789        1.0847  0.0197  50.4437
      4            0.6356        0.2310       0.6284            0.6284        1.6120  0.0192  50.5390
      5            0.7747        0.2025       0.7385            0.7385        0.5800  0.0187  50.5833
      6            0.9333        0.1900       0.9633            0.9633        0.1494  0.0179  50.5905
      7            0.6115 

In [ ]:
for subject in range(len(training_files)):
    training_function(subject)


 ######################### Training for Subject: 1 ######################### 

  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr     dur
-------  ----------------  ------------  ----------------  ------------  ------  ------
      1            0.5239        0.6587            0.5000        4.7591  0.0200  9.0020
      2            0.6517        0.3797            0.5074        2.9087  0.0199  8.2990
      3            0.7160        0.2881            0.5221        3.3227  0.0197  9.0790
      4            0.7647        0.2320            0.5257        2.4932  0.0192  8.5510
      5            0.9384        0.1507            0.7096        1.3055  0.0187  8.8320
      6            0.5653        0.1403            0.5000        4.3148  0.0179  8.6180
      7            0.9550        0.1484            0.7059        1.0289  0.0171  8.7350
      8            0.9789        0.2000            0.6618        1.0018  0.0161  8.4860
      9            0.9945        0.1222 

Best Cross Validation Kappa Score: 0.59

 ######################### Training for Subject: 4 ######################### 

  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr      dur
-------  ----------------  ------------  ----------------  ------------  ------  -------
      1            0.8695        0.5305            0.6471        2.0038  0.0200  18.9708
      2            0.5901        0.2305            0.5257        2.5732  0.0199  7.4970
      3            0.9862        0.1243            0.6250        2.6868  0.0197  7.2180
      4            0.9724        0.1167            0.6287        3.2727  0.0192  7.1670
      5            0.9853        0.0963            0.6838        1.9045  0.0187  7.4470
      6            0.9669        0.0662            0.6397        2.0989  0.0179  7.2070
      7            0.8290        0.1077            0.5368        5.1095  0.0171  7.7630
      8            1.0000        0.0818            0.6434        3.1889  0.0161  7.28

Best Cross Validation Kappa Score: 0.71

 ######################### Training for Subject: 7 ######################### 

  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr      dur
-------  ----------------  ------------  ----------------  ------------  ------  -------
      1            0.7031        1.0620            0.5735        1.0505  0.0200  27.2696
      2            0.5303        0.4247            0.4926        2.1601  0.0199  8.4800
      3            0.9145        0.3163            0.7574        0.5808  0.0197  9.3040
      4            0.5790        0.2331            0.5294        1.5632  0.0192  9.6570
      5            0.9449        0.2056            0.6140        1.2266  0.0187  7.8570
      6            0.7767        0.2326            0.6066        1.2998  0.0179  9.0460
      7            0.9237        0.1739            0.7904        0.4986  0.0171  8.8240
      8            0.9283        0.1647            0.6066        1.5549  0.0161  9.36